In [7]:
from pydantic_settings import BaseSettings

In [8]:
import os
from llama_index import ServiceContext, LLMPredictor, OpenAIEmbedding, PromptHelper
from llama_index.llms import OpenAI
from llama_index.text_splitter import TokenTextSplitter
from llama_index.node_parser import SimpleNodeParser
from llama_index import VectorStoreIndex, SimpleDirectoryReader
from llama_index import set_global_service_context
import tiktoken

In [9]:
from llama_index.node_parser import TokenTextSplitter
import together
import logging
from typing import Any, Dict, List, Mapping, Optional

from pydantic import Extra, Field, root_validator

from langchain.callbacks.manager import CallbackManagerForLLMRun
from langchain.llms.base import LLM
from langchain.llms.utils import enforce_stop_tokens
from langchain.utils import get_from_dict_or_env

In [10]:
from llama_index.embeddings import TogetherEmbedding
#import chromadb
#from llama_index.vector_stores import ChromaVectorStore
from llama_index.storage.storage_context import StorageContext


In [11]:
from llama_index.core import PromptTemplate
from IPython.display import Markdown, display

In [12]:
from llama_index.core import PromptTemplate

In [36]:
from llama_index.indices.postprocessor import SentenceTransformerRerank

In [13]:
from llama_index import SimpleDirectoryReader

documents = SimpleDirectoryReader("./ML-DS-TEXTBOOKS").load_data()

In [14]:
text_splitter = TokenTextSplitter(
  separator=" ",
  chunk_size=1024,
  chunk_overlap=20,
  backup_separators=["\n"],
  tokenizer=tiktoken.encoding_for_model("gpt-3.5-turbo").encode

)

nodes = text_splitter.get_nodes_from_documents(documents)

In [15]:
together.api_key = os.environ["TOGETHER_API_KEY"]

In [16]:
class TogetherLLM(LLM):
    """Together large language models."""

    model: str = "togethercomputer/llama-2-70b-chat"
    """model endpoint to use"""

    together_api_key: str = os.environ["TOGETHER_API_KEY"]
    """Together API Key"""

    temperature: float = 0.7
    """what sampling temperature to use. """

    max_tokens: int = 512
    """The maximum number of tokens to generate in the completion. """

    class Config:
        extra = Extra.forbid

    #root_validator()
    def validate_environment(cls, values: Dict) -> Dict:
        """Validate that the API key is set. """
        api_key = get_from_dict_or_env(
            values, "together_api_key", "TOGETHER_API_KEY"
        )
        values["together_api_key"] = api_key
        return values

    #property
    def _llm_type(self) -> str:
        """Rteurn type of LLM. """
        return "together"

    def _call(
        self,
        prompt: str,
        **kwargs: Any,
    ) -> str:
        """Call to Together endpoint. """
        together.api_key = self.together_api_key
        output =  together.Complete.create(prompt,
                                           model=self.model,
                                           max_tokens=self.max_tokens,
                                           temperature=self.temperature,
                                           )
        text = output['output']['choices'][0]['text']
        return text

C:\Users\j\AppData\Local\Temp\ipykernel_15640\630357882.py:17: PydanticDeprecatedSince20: `pydantic.config.Extra` is deprecated, use literal values instead (e.g. `extra='allow'`). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  extra = Extra.forbid


In [17]:
together_llm = TogetherLLM(
    model = "togethercomputer/llama-2-70b-chat",
    temperature = 0.1,
    max_tokens=512
)

In [18]:
llm = together_llm

embed_model = "local"

prompt_helper = PromptHelper(

  context_window=4096, 

  num_output=256, 

  chunk_overlap_ratio=0.1, 

  chunk_size_limit=None

)

In [19]:
service_context = ServiceContext.from_defaults(

  llm=llm,

  embed_model=embed_model,

  node_parser=text_splitter,

  prompt_helper=prompt_helper

)

c:\Users\j\python\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [20]:
index = VectorStoreIndex.from_documents(
    documents, 
    service_context = service_context
    #storage_context=storage_context
)

In [40]:
# Initialize the reranker
rerank = SentenceTransformerRerank(
    model="cross-encoder/ms-marco-MiniLM-L-2-v2", top_n=3 # Note here
)

config.json: 100%|██████████| 794/794 [00:00<00:00, 57.9kB/s]
pytorch_model.bin: 100%|██████████| 62.5M/62.5M [00:27<00:00, 2.29MB/s]
tokenizer_config.json: 100%|██████████| 316/316 [00:00<00:00, 18.5kB/s]
vocab.txt: 100%|██████████| 232k/232k [00:00<00:00, 890kB/s]
special_tokens_map.json: 100%|██████████| 112/112 [00:00<00:00, 4.26kB/s]


In [41]:
#query_engine = index.as_query_engine(similarity_top_k=2, llm=llm)
query_engine = index.as_query_engine(similarity_top_k=5, node_postprocessors=[rerank], llm=llm)
vector_retriever = index.as_retriever(similarity_top_k=5)

In [42]:
template = (
    "We have provided context information below. \n"
    "---------------------\n"
    "{context_str}"
    "\n---------------------\n"
    "Given the context information and not prior knowledge, "
    "You are a helpful Machine Learning assistant. Given a query follow the following steps: think through the question carefully and in detail, split the question into sub-questions, solve the first sub-question, move on and solve the next sub-question, When you have solved all sub-questions combine the knowledge and information gotten, and solve the main question. Your response/answer should be conprehensive and not contradicted with the following context if they are relevant, otherwise ignore them if they are not relevant and . If you can not find any good answer \n"
    "Query:  {query_str}\n"
    "Answer: "
)

In [43]:
qa_template = PromptTemplate(template)
query_engine.update_prompts(
    {"response_synthesizer:text_qa_template": qa_template}
)

In [44]:
query_str = "What  is  repeated  cross-validation  and  why  would  you  prefer  it  to  using  a  single validation set?"

In [45]:
response = query_engine.query(query_str)
print(str(response))

 Repeated cross-validation is a technique used in machine learning to evaluate the performance of a model by training and testing it multiple times on different subsets of the data. In contrast to using a single validation set, repeated cross-validation provides a more robust estimate of a model's performance by averaging the performance over multiple iterations.

The main advantage of repeated cross-validation is that it reduces the variance in the performance estimate compared to using a single validation set. With a single validation set, the performance of the model can be heavily influenced by the specific samples in the set, leading to a biased estimate of the model's true performance. Repeated cross-validation helps to mitigate this issue by averaging the performance over multiple iterations, providing a more reliable estimate of the model's performance.

Another advantage of repeated cross-validation is that it allows for a more thorough exploration of the hyperparameter space.

Evaluation